In [129]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [130]:
train=pd.read_csv("imdb_reviews.csv")
test=pd.read_csv("test_reviews.csv")

In [131]:
train.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [132]:
test.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


In [133]:
index=pd.read_csv("word_indexes.csv")

In [134]:
index.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


In [135]:
index=dict(zip(index.Words,index.Indexes))
index["<PAD"]=0
index["<START"]=1
index["<UNK>"]=2
index["<UNUSED>"]=3

In [136]:
def review_encoder(text):
  arr=[index[word] for word in text]
  return arr

In [137]:
train_data,train_labels=train['Reviews'],train['Sentiment']
test_data, test_labels=test['Reviews'],test['Sentiment']
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [138]:
train_data.head()

0    [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...
1    [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...
2    [1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...
3    [1, 4, 2, 2, 33, 2804, 4, 2040, 432, 111, 153,...
4    [1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...
Name: Reviews, dtype: object

In [139]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

In [140]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=0,padding='post',maxlen=256)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=0,padding='post',maxlen=256)

In [141]:
model = keras.Sequential([
    keras.layers.Embedding(10000, 16, input_length=256),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [142]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [143]:
#training the model
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 2s 25ms/step - loss: 0.6913 - accuracy: 0.6127 - val_loss: 0.6880 - val_accuracy: 0.6332
Epoch 2/30
49/49 [==============================] - 1s 23ms/step - loss: 0.6804 - accuracy: 0.6992 - val_loss: 0.6705 - val_accuracy: 0.7188
Epoch 3/30
49/49 [==============================] - 1s 20ms/step - loss: 0.6506 - accuracy: 0.7668 - val_loss: 0.6310 - val_accuracy: 0.7647
Epoch 4/30
49/49 [==============================] - 1s 20ms/step - loss: 0.5968 - accuracy: 0.7949 - val_loss: 0.5726 - val_accuracy: 0.7973
Epoch 5/30
49/49 [==============================] - 1s 21ms/step - loss: 0.5275 - accuracy: 0.8254 - val_loss: 0.5058 - val_accuracy: 0.8224
Epoch 6/30
49/49 [==============================] - 1s 23ms/step - loss: 0.4564 - accuracy: 0.8496 - val_loss: 0.4462 - val_accuracy: 0.8408
Epoch 7/30
49/49 [==============================] - 1s 22ms/step - loss: 0.3979 - accuracy: 0.8678 - val_loss: 0.4015 - val_accuracy: 0.8509
Epoch 8/30
49

In [144]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 2s 2ms/step - loss: 0.3124 - accuracy: 0.8798


In [146]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, 'tfjs_model')
